# Select database (i.e. staging, production, local authenticated, local unauthenticated)

In [1]:
import requests
from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.common import ProjectContext
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    LOCAL_AUTHENTICATED = "local_authenticated"

obi_one_api_url = "http://127.0.0.1:8100"
virtual_lab_id="edb1fb49-660b-477b-af7b-12913b2df7d9"
project_id="d4d004bc-44c5-4f15-94a7-1e4478e82b1d"

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED
# database_environment = DatabaseEnvironment.LOCAL_AUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local":
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local":
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "https://127.0.1:8000/api/entitycore"

elif DatabaseEnvironment.LOCAL_AUTHENTICATED:
    """
    Not yet tested.
    - Change APP_DISABLE_AUTH to False in entitycore/.env.run-local
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk morphology.ipynb example)
    """
    entitycore_api_url = "https://127.0.1:8000/api/entitycore"

else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

# Fetch a morphology

In [2]:
from entitysdk.models.morphology import (
    ReconstructionMorphology,
)
kwargs = {}
reconstruction_morphologies = client.search_entity(
    entity_type=ReconstructionMorphology, query=kwargs, limit=10
).all()

reconstruction_morphology_id = reconstruction_morphologies[0].id
print(reconstruction_morphology_id)

393b267f-7bd8-4569-be1c-968544e52729


# Call the declared morphology metrics endpoint

In [3]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/neuron-morphology-metrics/{reconstruction_morphology_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


Success: {'aspect_ratio': 0.4873803958856252, 'circularity': 0.6788194705016894, 'length_fraction_above_soma': 0.4292110204696655, 'max_radial_distance': 930.7633666992188, 'number_of_neurites': 5, 'soma_radius': 10.585715293884277, 'soma_surface_area': 1408.1544189453125}


# Call the generated morphology metrics endpoint

In [4]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/morphology-metrics-run-grid"

# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request body — adjust this with the actual morphology metrics form data!
# Placeholder example:
request_body = {
  "type": "MorphologyMetricsForm",
  "initialize": {
    "type": "MorphologyMetricsForm.Initialize",
    "morphology": {
      "id_str": str(reconstruction_morphology_id),
    }
  }
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

Success: {'0': {'aspect_ratio': 0.4873803958856252, 'circularity': 0.6788194705016894, 'length_fraction_above_soma': 0.4292110204696655, 'max_radial_distance': 930.7633666992188, 'number_of_neurites': 5, 'soma_radius': 10.585715293884277, 'soma_surface_area': 1408.1544189453125}}


In [7]:
from entitysdk.models.electrical_cell_recording import (
    ElectricalCellRecording,
)
kwargs = {"name": "S1FL_L5_DBC_cIR_4"}
Electrical_cell_recording = client.search_entity(
    entity_type=ElectricalCellRecording, query=kwargs, limit=1
).one()

Electrical_cell_recording_id = Electrical_cell_recording.id
print(Electrical_cell_recording_id)

f3948959-9d55-4fef-840d-b6b7e36e92e1


# Call the declared electrophysiology recording metrics endpoint

In [8]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/electrophysiologyrecording-metrics/{Electrical_cell_recording_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


Success: {'feature_dict': {'step_0': {'spike_count': {'avg': 1.6666666666666667, 'num_traces': 3, 'unit': None}, 'time_to_first_spike': {'avg': 6.62500000002413, 'num_traces': 3, 'unit': 'ms'}, 'time_to_last_spike': {'avg': 19.400000000027035, 'num_traces': 3, 'unit': 'ms'}, 'inv_time_to_first_spike': {'avg': 303.1065437050581, 'num_traces': 3, 'unit': 'Hz'}, 'doublet_ISI': {'avg': 19.162500000004357, 'num_traces': 2, 'unit': 'ms'}, 'inv_first_ISI': {'avg': 72.74608153443701, 'num_traces': 2, 'unit': 'Hz'}, 'mean_frequency': {'avg': 228.74223751569082, 'num_traces': 3, 'unit': 'Hz'}, 'strict_burst_number': {'avg': 0.0, 'num_traces': 3, 'unit': None}, 'AP_height': {'avg': -12.364847343260458, 'num_traces': 3, 'unit': 'mV'}, 'AP_amplitude': {'avg': 50.82487408104208, 'num_traces': 3, 'unit': 'mV'}, 'AP1_amp': {'avg': 52.47725295811486, 'num_traces': 3, 'unit': 'mV'}, 'APlast_amp': {'avg': 49.1724952039693, 'num_traces': 3, 'unit': 'mV'}, 'AP_duration_half_width': {'avg': 0.30833333333340